Imports

In [2]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from time import sleep
from selenium.common.exceptions import NoSuchElementException 

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor
import threading

In [2]:
# Instantiate options
options = Options()

# Add extra options
options.add_argument("--disable-infobars")  # Disable the infobars
options.add_argument("--disable-popup-blocking")  # Disable pop-ups
options.add_argument("--ignore-certificate-errors")  # Ignore certificate errors
options.add_argument("--incognito")  # Use Chrome in incognito mode

Facebook

In [10]:
import pandas as pd

# Creating dataframe to store all posts' information
facebook_captions_df = pd.DataFrame(columns=["Source","Account","Caption","Image"])

In [11]:
# Usernames of accounts that will be scraped
# Increase the list to scrape more pages

# Each item in the list should be the text written in the second part of the page's URL
# Example: https://www.facebook.com/OrangeJordan/

facebook_pages = ["OrangeJordan", "TED", "PSUTOFFICIAL"]

In [12]:
# Function to store each post's details in the dataframe

def scrape_facebook_post(posts,page,df):
    driver = webdriver.Chrome(options=options)

    for p in posts:
        driver.get(p)

        # Exit login pop-up menu
        button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[aria-label="Close"][role="button"]'))).click()
        
        # Click see more
        try:
            see_more = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//div[@role="button" and text()="See more"]'))).click()
        except:
            pass
    
        image_elements = driver.find_elements(By.TAG_NAME, "img") 
        image = image_elements[0].get_attribute('src') # Get post image

        caption = driver.find_element(By.CSS_SELECTOR, 'div.xyinxu5.x4uap5.x1g2khh7.xkhd6sd').text # Get post caption
        
        # Add details of post into dataframe
        new_row = {"Source": "Facebook", "Account": page, "Caption": caption, "Image": image}
        df.loc[len(df)] = new_row

In [13]:
# Function to handle each Facebook page

def scrape_facebook_page(page, df):
    # open the webpage
    driver = webdriver.Chrome(options=options)
    driver.get("http://www.facebook.com")

    # Target username
    username = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
    password = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

    # Enter username and password
    username.clear()
    username.send_keys("username")
    password.clear()
    password.send_keys("password")

    # Target the login button and click it
    button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
    sleep(5)

    # Open account page
    driver.get("https://www.facebook.com/" + page + "/photos_by")
    sleep(5)

    # scroll down
    # The bigger the range, the more you scroll down
    for scroll in range(0, 1):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(10)

    # Collect URLs of posts in photos section
    anchors = driver.find_elements(By.TAG_NAME, 'a')
    anchors = [a.get_attribute('href') for a in anchors]
    anchors = [a for a in anchors if str(a).startswith("https://www.facebook.com/photo")]

    driver.quit()

    # Use ThreadPoolExecutor to scrape posts concurrently
    # Increasing number of anchors, increases number of rows added to dataframe
    with ThreadPoolExecutor() as exe:
        exe.map(lambda post: scrape_facebook_post([post], page, df), anchors[:8])


In [14]:
# Multiprocessing to scrape multiple Facebook pages concurrently
# Run the scraping process in parallel for each page

with ThreadPoolExecutor() as exe:
    exe.map(lambda page: scrape_facebook_page(page, facebook_captions_df), facebook_pages)

In [15]:
# Display facebook_captions_df
facebook_captions_df

,Source,Account,Caption,Image
0,Facebook,PSUTOFFICIAL,تتقدم أسرة جامعة الأميرة سمية للتكنولوجيا، ممث...,https://scontent.famm6-1.fna.fbcdn.net/v/t39.3...
1,Facebook,TED,The most popular TED Talks of 2023 are here! F...,https://scontent.famm6-1.fna.fbcdn.net/v/t39.3...
2,Facebook,TED,Become a TED Member to help support our missio...,https://scontent.famm6-1.fna.fbcdn.net/v/t39.3...
3,Facebook,TED,How often do you think about your own death? I...,https://scontent.famm6-1.fna.fbcdn.net/v/t39.3...
4,Facebook,PSUTOFFICIAL,شاركت جامعة الأميرة سمية للتكنولوجيا في جناح ا...,https://scontent.famm6-1.fna.fbcdn.net/v/t39.3...
5,Facebook,OrangeJordan,لجميع مشتركي Orange موبايل\nاحكي مع أثيوبيا بـ...,https://scontent.famm6-1.fna.fbcdn.net/v/t39.3...
6,Facebook,TED,The life-long bond between brothers and sister...,https://scontent.famm6-1.fna.fbcdn.net/v/t39.3...
7,Facebook,TED,Don't wait — let the people you love know how ...,https://scontent.famm6-1.fna.fbcdn.net/v/t39.3...
8,Facebook,OrangeJordan,The Apple Event is coming up soon! Tune in liv...,https://scontent.famm6-1.fna.fbcdn.net/v/t39.3...
9,Facebook,OrangeJordan,,https://scontent.famm6-1.fna.fbcdn.net/v/t39.3...


Instagram

In [3]:
import pandas as pd

# Creating dataframe to store all posts' information
instagram_captions_df = pd.DataFrame(columns=["Source","Account","Caption","Image"])

In [4]:
# Usernames of accounts that will be scraped
# Increase the list to scrape more pages

# Each item in the list should be the username of the page
# In case of hashtags, write the exact hashtag you are looking for

instagram_pages = ["orangejo","psutofficial","#orange", "#apple", "#banana"]

In [5]:
# Function to store each post's details in the dataframe

def scrape_instagram_post(posts,page,df):
    driver = webdriver.Chrome(options=options)
    
    #follow each image link and extract only image at index=1
    for p in posts:
        driver.get(p)
        sleep(5)
        
        img = driver.find_elements(By.TAG_NAME, 'img') # finding image element
        img = [i.get_attribute('src') for i in img]
        image = img[2] # extract image out of element

        # get caption
        caption = driver.find_element(By.CSS_SELECTOR, "h1[dir=auto]").text

    # Add details of post into dataframe
    new_row = {"Source": "Instagram", "Account": page, "Caption": caption, "Image": image}
    df.loc[len(df)] = new_row

In [6]:
# Function to handle each instagram page/hashtag

def scrape_instagram_page(page, df):
    #specify the path to chromedriver.exe (download and save on your computer)
    driver = webdriver.Chrome()

    #open the webpage
    driver.get("http://www.instagram.com")

    #target username
    username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
    password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

    #enter username and password
    username.clear()
    username.send_keys("username")
    password.clear()
    password.send_keys("password")

    #target the login button and click it
    button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

    #We are logged in!

    # Dismiss pop up
    sleep(2)
    popup1 = WebDriverWait(driver, 7).until(EC.element_to_be_clickable((By.XPATH, "//div[@role='button' and text()='Not now']"))).click()
    popup2 = WebDriverWait(driver, 7).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Not Now")]'))).click()

    #click on the search and explore icon
    search_icon = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div[class='x1n2onr6 x6s0dn4 x78zum5'] a[role='link'][href='#']"))).click()

    #target the search input field
    searchbox = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[aria-label='Search input']")))
    searchbox.clear()

    #search for the page
    searchbox.send_keys(page)

    #dealing with the hashtag in case we are searching for a hashtag
    if page[0] == "#":
        page = page[1:]

    # opening on the page
    sleep(2) 
    my_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, '/" + page + "/')]"))).click()

    #scroll down 2 times
    #increase the range to scroll more
    n_scrolls = 2
    for j in range(0, n_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(5)
        
    #target all the link elements on the page
    anchors = driver.find_elements(By.TAG_NAME, 'a')
    anchors = [a.get_attribute('href') for a in anchors]
    #narrow down all links to image links only
    anchors = [a for a in anchors if str(a).startswith("https://www.instagram.com/p/")]

    driver.quit()

    # Use ThreadPoolExecutor to scrape posts concurrently
    # Increasing number of anchors, increases number of rows added to dataframe
    with ThreadPoolExecutor() as exe:
        exe.map(lambda post: scrape_instagram_post([post], page, df), anchors[:8])

In [35]:
# Multiprocessing to scrape multiple Instagram pages/hashtags concurrently
# Run the scraping process in parallel for each page

def scrape_with_dynamic_delay(page, df, index, delay=6):
    sleep(index * delay)  # Delay based on the index (first one runs instantly)
    scrape_instagram_page(page, df)

with ThreadPoolExecutor() as executor:
    for index, page in enumerate(instagram_pages):
        # Delay each webscraping attempt to prevent being blocked at sign in
        executor.submit(scrape_with_dynamic_delay, page, instagram_captions_df, index)

In [34]:
# Displaying instagram_captions_df
instagram_captions_df

,Source,Account,Caption,Image


Pinterest